In [3]:
# change the model name and similarity metric to the model you want to use
model_name="blip"
sim_metric='ssim' #'llava' or 'blip' or 'ssim'
####################################

base_path = "output/model"
constraints = {
    "Positive": "Sentiment/Positive/constrained_eps_32_batch_8",
    "Negative": "Sentiment/Negative/constrained_eps_32_batch_8",
    "Neutral": "Sentiment/Neutral/constrained_eps_32_batch_8",
    "Formal": "Formality/Formal/constrained_eps_32_batch_8",
    "Informal": "Formality/Informal/constrained_eps_32_batch_8",
    "English": "Language/English/constrained_eps_32_batch_8",
    "French": "Language/French/constrained_eps_32_batch_8",
    "Spanish": "Language/Spanish/constrained_eps_32_batch_8",
    "Left": "Politics/Left/constrained_eps_32_batch_8",
    "Right": "Politics/Right/constrained_eps_32_batch_8",
    "Spam": "Attack/Spam/constrained_eps_32_batch_8",
    "Injection": "Attack/Injection/constrained_eps_32_batch_8"
}

file_dict = {}
for key, constraint in constraints.items():
    # First add paths with format base_path/i/constraint/
    paths = [f"{base_path}/{i}/{constraint}/" for i in range(0,5)]
    # Then add paths with format base_path/coco_i/constraint/
    paths.extend([f"{base_path}/coco_{i}/{constraint}/" for i in range(1,11)])
    file_dict[key] = paths


if model_name == 'minigpt4':
    for category, paths in file_dict.items():
        file_dict[category] = [path.replace("model", model_name) for path in paths]
elif model_name == 'llava':
    for category, paths in file_dict.items():
        file_dict[category] = [path.replace("model", model_name) for path in paths]
elif model_name == 'blip':
    for category, paths in file_dict.items():
        file_dict[category] = [path.replace("model", model_name) for path in paths]

# Binary classfication: To check if the model's output matches the label/content of the image

In [ ]:
import json
import numpy as np

def get_classification_result(file_path):
    outputs = []
    with open(file_path, 'r') as f:
        for line in f:
            data = json.loads(line)
            if 'output' in data:
                outputs.append(data['output'])
    
    yes_count = sum(1 for o in outputs if "yes" in o.lower() or "oui" in o.lower())
    total = len(outputs)
    return yes_count/total if total > 0 else 0

classification_dict = {}
for key in file_dict:
    results = [get_classification_result(f"{file}content_classification_result.jsonl") for file in file_dict[key]]
    classification_dict[key] = {
        "mean": round(np.mean(results), 2),
        "std": round(np.std(results), 2)
    }

# Calculate category means
categories = {
    "sentiment": ["Positive", "Negative", "Neutral"],
    "language": ["English", "French", "Spanish"],
    "formality": ["Formal", "Informal"],
    "politics": ["Left", "Right"],
    "Attack": ["Spam", "Injection"]
}

for cat, keys in categories.items():
    means = [classification_dict[k]["mean"] for k in keys]
    stds = [classification_dict[k]["std"] for k in keys]
    classification_dict[cat] = {
        "mean": round(sum(means)/len(means), 2),
        "std": round(np.sqrt(sum(s**2 for s in stds)/len(stds)), 2)
    }

classification_dict

# Similarity between oringinal and perturbed image
## Model Initialization

In [ ]:
# Model Initialization and Embedding Calculation
import torch
from PIL import Image

if model_name == "llava":
    import argparse
    from llava_llama_2.utils import get_model
    
    def parse_args():
        parser = argparse.ArgumentParser(description="Demo")
        parser.add_argument("--model_path", type=str, default="./ckpts/llava_llama_2_13b_chat_freeze")
        parser.add_argument("--gpu_id", type=int, default=0, help="specify the gpu to load the model.")
        parser.add_argument("--model_base", type=str, default=None)
        return parser.parse_args(args=['--gpu_id', '0'])

    args = parse_args()
    tokenizer, model, image_processor, model_name = get_model(args)
    model.eval()

elif model_name == "minigpt4":
    import argparse
    from minigpt4.common.config import Config
    from minigpt4.common.registry import registry
    from minigpt_utils import prompt_wrapper, generator
    
    def parse_args():
        parser = argparse.ArgumentParser(description="Demo")
        parser.add_argument("--cfg-path", default="eval_configs/minigpt4_eval.yaml", help="path to configuration file.")
        parser.add_argument("--gpu-id", type=int, default=0, help="specify the gpu to load the model.")
        parser.add_argument("--mode", type=str, default='VisualChatBot', choices=["TextOnly", "VisualChatBot"])
        parser.add_argument("--image_file", type=str, default='./image.bmp', help="Image file")
        parser.add_argument("--output_file", type=str, default='./result.jsonl', help="Output file.")
        parser.add_argument("--options", nargs="+")
        return parser.parse_args(args=['--cfg-path', 'eval_configs/minigpt4_eval.yaml', 
                                     '--gpu-id', '0', '--mode', 'VisualChatBot', 
                                     '--image_file', 'clean_images/0.png', 
                                     '--output_file', './result.jsonl'])
    args = parse_args()
    cfg = Config(args)
    model_config = cfg.model_cfg
    model_config.device_8bit = args.gpu_id
    model = registry.get_model_class(model_config.arch).from_config(model_config).to(f'cuda:{args.gpu_id}')
    vis_processor = registry.get_processor_class(cfg.datasets_cfg.cc_sbu_align.vis_processor.train.name).from_config(cfg.datasets_cfg.cc_sbu_align.vis_processor.train)
    my_generator = generator.Generator(model=model)

elif model_name == "blip":
    import argparse
    from lavis.models import load_model_and_preprocess
    
    def parse_args():
        parser = argparse.ArgumentParser(description="Demo")
        parser.add_argument("--gpu-id", type=int, default=0, help="specify the gpu to load the model.")
        parser.add_argument("--data_path", type=str, default="instruction_data/0/Sentiment/dataset.csv")
        parser.add_argument("--image_file", type=str, default='./image.bmp', help="Image file")
        parser.add_argument("--output_file", type=str, default='./result.jsonl', help="Output file.")
        parser.add_argument("--instruction", type=str, default=None, choices=["positive", "negative", "neutral", "irony", "non_irony", "formal", "informal", "french", "english", "spanish", "left", "right", "inference_content_evaluation", "injection", "spam"])
        parser.add_argument("--image_index", type=int, default=0)
        return parser.parse_args(args=['--data_path', 'instruction_data/0/Attack/dataset.csv',
                                     '--image_file', 'clean_images/0.png',
                                     '--output_file', './result.jsonl'])

    args = parse_args()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model, vis_processor, _ = load_model_and_preprocess(
        name='blip2_vicuna_instruct',
        model_type='vicuna13b',
        is_eval=True,
        device=device
    )
    model.eval()
    img = Image.open(args.image_file).convert('RGB')
    img = vis_processor["eval"](img).unsqueeze(0).to(device)


In [ ]:
from PIL import Image
from torchvision import transforms
import torch
import matplotlib.pyplot as plt
from pytorch_msssim import ssim

def denormalize(images):
    mean = torch.tensor([0.48145466, 0.4578275, 0.40821073]).cuda()
    std = torch.tensor([0.26862954, 0.26130258, 0.27577711]).cuda()
    return images * std[None, :, None, None] + mean[None, :, None, None]

def load_image(image_path):
    return Image.open(image_path).convert('RGB')

def calculate_similarity(image1, image2, model_name, model, sim_metric):
    cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
    
    # Process images if not already tensors
    if not isinstance(image1, torch.Tensor):
        if model_name == 'llava':
            image1 = image_processor.preprocess(image1, return_tensors='pt')['pixel_values'].cuda()
            image2 = image_processor.preprocess(image2, return_tensors='pt')['pixel_values'].cuda()
        elif model_name == 'minigpt4':
            image1 = [vis_processor(image1).unsqueeze(0).to(model.device)]
            image2 = [vis_processor(image2).unsqueeze(0).to(model.device)]
        elif model_name == 'blip':
            image1 = vis_processor["eval"](image1).unsqueeze(0).to(device)
            image2 = vis_processor["eval"](image2).unsqueeze(0).to(device)
    
    # Get embeddings based on model type
    if sim_metric == 'llava':
        emb1 = model.encode_images(image1.half())
        emb2 = model.encode_images(image2.half())
    elif sim_metric == 'minigpt4':
        prompt1 = prompt_wrapper.Prompt(model=model, img_prompts=[[image1]])
        prompt2 = prompt_wrapper.Prompt(model=model, img_prompts=[[image2]])
        emb1, emb2 = prompt1.img_embs[0][0], prompt2.img_embs[0][0]
    elif sim_metric == 'blip':
        with model.maybe_autocast():
            emb1 = model.ln_vision(model.visual_encoder(image1))
            emb2 = model.ln_vision(model.visual_encoder(image2))
    elif sim_metric == 'ssim':
        return ssim(image1, image2, data_range=1.0, size_average=True)
    return cos(emb1.view(-1).to(torch.float32), emb2.view(-1).to(torch.float32))

def calculate_transformations_sim(image, model_name, model, sim_metric):
    transforms_list = [
        transforms.GaussianBlur(9, sigma=(1.0, 5.0)),
        transforms.RandomAffine(45),
        transforms.ColorJitter(0.5, 0.5, 0.5, 0.2),
        transforms.RandomHorizontalFlip(1),
        transforms.RandomPerspective(0.5, 1)
    ]
    # Process image based on model
    if model_name == 'llava':
        image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'].cuda()
    elif model_name == 'minigpt4':
        image_tensor = vis_processor(image).unsqueeze(0).cuda()
    elif model_name == 'blip':
        image_tensor = vis_processor["eval"](image).unsqueeze(0).cuda()

    # Calculate similarities for each transformation
    sims = [calculate_similarity(
        denormalize(image_tensor), 
        denormalize(transform(image_tensor)),
        model_name, model, sim_metric
    ) for transform in transforms_list]
    
    return sum(sims)/len(sims)

In [ ]:
sim_dict = {}
sim_metric='ssim'
# Calculate random pairs similarity
temp_list = [calculate_similarity(load_image(f"clean_images/coco_{i}.jpg"), 
                                load_image(f"clean_images/coco_{j}.jpg"), 
                                model_name, model, sim_metric)
             for i in range(1,10) for j in range(i+1,11)]
sim_dict['random pairs'] = {'mean': round(torch.mean(torch.stack(temp_list)).item(), 3),
                           'std': round(torch.std(torch.stack(temp_list)).item(), 3)}

# # Calculate similarity between clean and their augmentations
temp_list = [calculate_transformations_sim(load_image(f"clean_images/coco_{i}.jpg"),
                                         model_name, model, sim_metric)
             for i in range(1,11)]
sim_dict['augmentations'] = {'mean': round(torch.mean(torch.stack(temp_list)).item(), 3),
                            'std': round(torch.std(torch.stack(temp_list)).item(), 3)}

# Calculate similarity between clean and perturbed images
for key in file_dict:
    temp_list = [calculate_similarity(load_image(f"{file}clean_prompt.bmp"),
                                    load_image(f"{file}bad_prompt.bmp"),
                                    model_name, model, sim_metric)
                 for file in file_dict[key]]
    sim_dict[key] = {'mean': round(torch.mean(torch.stack(temp_list)).item(), 3),
                     'std': round(torch.std(torch.stack(temp_list)).item(), 3)}
print(sim_dict)

In [ ]:
import numpy as np

def calc_stats(means, stds, n=15):
    mean_avg = np.mean(means)
    S_within = np.sum((n-1) * stds**2)
    S_between = np.sum(n * (means - mean_avg)**2)
    var_avg = (S_within + S_between) / (len(means)*n - 1)
    return {'mean': mean_avg, 'std': np.sqrt(var_avg)}

result = {task: calc_stats(
    np.array([sim_dict[k]['mean'] for k in keys]),
    np.array([sim_dict[k]['std'] for k in keys])
) for task, keys in categories.items()}

overall = calc_stats(
    np.array([r['mean'] for r in result.values()]),
    np.array([r['std'] for r in result.values()])
)

for task in result:
    print(f"{task}: mean = {result[task]['mean']:.3f}, std = {result[task]['std']:.3f}")
print(f"\nOverall: mean = {overall['mean']:.3f}, std = {overall['std']:.3f}")